In [ ]:
############################################################################################
############################################################################################
   ###################### Desk Reject Project: Creating final model with parsed down words
############################################################################################
############################################################################################
#################################################
  ###### SQL request to get data; then put data in pandas df.
#################################################

############################
  #### Get data from today until 7 days ago
############################
from datetime import datetime, timedelta
start_date = str(datetime.date(datetime.now()) - timedelta(days=7))
end_date = str(datetime.date(datetime.now()))




############################
  #### Make a connection to the EM data source and execute the query:
############################
import pymssql
import pandas as pd
conn = pymssql.connect(server='*WITHHELD*', user='*WITHHELD*', password='*WITHHELD*', database='*WITHHELD*')
    # NOTE: database='PONE' is not needed but seems to speed up the query.

####### Make a call on the source with a specific SQL querry
cur = conn.cursor(as_dict=True)     # 'as_dict' allow you to transform the query into a pandas df
cur.execute(
    f"""
    SELECT d.PUBDNUMBER, d.DOCUMENTID, d.DTITLE, d.SHORT_TITLE, d.ABSTRACT_TEXT, d.ORIGINAL_SUBMISSION_START_DATE,
        d.CATEGORY, d.ENTER_KEYWORDS, d.REQUEST_EDITOR, d.SUGGEST_REVIEWERS, d.OPPOSE_REVIEWERS,
        a.DEPARTMENT, a.INSTITUTE, a.COUNTRY, a.PTITLE, a.POSITION, a.ALLAUTHORS, 
        a.ALL_AUTHOR_CONTRIBUTOR_ROLES, a.SECTIONNAME, a.FUNDER_NAME
    FROM pone.dbo.DOCUMENT AS d
    LEFT JOIN pone.dbo.ROLEAUTH_DOC_PEOPLE_ADDR AS a on d.DOCUMENTID = a.DOCUMENTID
    WHERE d.ORIGINAL_SUBMISSION_START_DATE >= '{start_date} 00:00:00.000' AND d.ORIGINAL_SUBMISSION_START_DATE <= '{end_date} 00:00:00.000' AND d.PUBDNUMBER IS NOT NULL
    """)

############################
  #### Put the data into a pandas df
############################
data = []
for i in range(9000000):
    if cur.fetchone() is None:
        break
    else:
        row = cur.fetchone()
        data.append(row)

data = pd.DataFrame(data[0:len(data)-1])

####### Print to confirm
data





In [ ]:
############################
  #### Save a copy of your data so you don't have to re-get the SQL:
############################
saved_copy_of_data = data.copy()

############################
  #### Load the copy of your saved initial data:
############################
#data = saved_copy_of_data



## Create parsed down free text variables:

In [ ]:
#################################################
  ###### USING COUNT VECTOR TO CREATE WORD/BI & TRI GRAM FREQUENCIES:
    # NOTE: Counts are better than TF-IDFs in this case: Same model quality metrics but tf-idfs require re-uploading all other values, word counts do not
       # See supplmental code: tf-idfs may be better for other projects.
#################################################
#####################
 # Create a sparse matrix of all the tf-idfs. Variable by variable for the following variables:
  ## DTITLE
  ## SHORT_TITLE
  ## ABSTRACT_TEXT
  ## DEPARTMENT
  ## INSTITUTE
  ## PTITLE
  ## ALL_AUTHOR_CONTRIBUTOR_ROLES
  ## FUNDER_NAME
#####################
###############
 # The following is a count vectorizer: This gives a frequency of words, bigrams and trigrams (with ngram_range as 1,3)
    # ALSO: a word or bi-gram or tri-gram in less than 2 docs or more than 80% of docs is removed).
    # INCLUDES stopwords
###############
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix = vect.fit_transform(data['DTITLE'].astype(str))
vect_2 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_2 = vect_2.fit_transform(data['SHORT_TITLE'].astype(str))
vect_3 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_3 = vect_3.fit_transform(data['ABSTRACT_TEXT'].astype(str))
vect_4 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_4 = vect_4.fit_transform(data['DEPARTMENT'].astype(str))
vect_5 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_5 = vect_5.fit_transform(data['INSTITUTE'].astype(str))
vect_6 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_6 = vect_6.fit_transform(data['PTITLE'].astype(str))
vect_7 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_7 = vect_7.fit_transform(data['ALL_AUTHOR_CONTRIBUTOR_ROLES'].astype(str))
vect_8 = CountVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 3))
doc_term_matrix_8 = vect_8.fit_transform(data['FUNDER_NAME'].astype(str))




## Parse down the word & bi/tri gram features: This is only a partial list of words due to confidentiality. I used around 250 words across these 8 fields in total

In [ ]:
#################################################
  ###### Parse down the word & bi/tri gram features:
    ### NOTE: THIS CELL IS *NOT* FOR THE INTIIAL MODEL BUT *AFTER* YOU FIND THE MOST RELEVANT WORDS FOR THE FINAL MODEL
#################################################
#####################
 # Function to parse down sparse matrix and create a parsed down dataframe to identify which index is which word/gram
    ## This also sorts the parsed list in the intended order (e.g. the order of dtitle_list).
    ## This also adds in words that CountVectorizer does pick up because they don't exist anywhere in the data (adds as all 0s)
#####################
def parse_sparse_matrix(word_list, vect_name, doc_term_matrix_name, main_var_name_in_quotes):
    ### Get the index of the words of interest
    aa = pd.DataFrame(vect_name.get_feature_names())
    new_feature_names = aa[aa[0].isin(word_list)]
    new_feature_names['main_var'] = main_var_name_in_quotes
    bb = list(new_feature_names.index)
    new_matrix = doc_term_matrix_name.tocsc()[:,bb]
    new_matrix = pd.DataFrame.sparse.from_spmatrix(new_matrix)         ## NEW: Turn sparse matrix into pandas df
    new_feature_names = new_feature_names.reset_index(drop = True)
    ##### You need to change the order of the columns to reflect the original list order:
    cc = []
    dd = []
    for i in word_list:
        if i in list(new_feature_names.iloc[:,0]):
            cc.append(i)
            dd.append(len(cc)-1)
    sorter = pd.DataFrame({'sort_values': cc, 'sort_num': dd})
    sorter = pd.merge(new_feature_names, sorter, left_on= 0, right_on = 'sort_values', how='left', sort=False)
    sorter['initial_wrong_order'] = sorter.index
    sorter = sorter.sort_values(by=['sort_num'])
    ## Sort new_matrix
    new_matrix = new_matrix[list(sorter['initial_wrong_order'])]
    ## new_feature_names is the sorter minus the 3 colmns that sorted it:
    new_feature_names = sorter.drop(['sort_values','sort_num', 'initial_wrong_order'], axis=1)
    ##### If a word never shows up in the data, CountVect skips the column completely; You'll need to add it (all 0s)
    if len(new_feature_names) != len(word_list):
        for i in range(len(word_list)):
            if len(new_feature_names) <= i:                            ## If need to add words to the end of the dfs
                ### 1. Insert the missed row in new_feature_names:
                line = pd.DataFrame({0: word_list[i], 'main_var': main_var_name_in_quotes}, index=[i])
                new_feature_names = pd.concat([new_feature_names.iloc[:i], line]).reset_index(drop=True)
                ### 2. Insert the missed column into new_matrix:
                new_matrix['new_'+ str(i)] = 0
            elif new_feature_names.iloc[i,0] != word_list[i]:            ## If the word isn't in the feature list, create it in both dfs
                ### 1. Insert the missed row in new_feature_names:
                line = pd.DataFrame({0: word_list[i], 'main_var': main_var_name_in_quotes}, index=[i])
                new_feature_names = pd.concat([new_feature_names.iloc[:i], line, new_feature_names.iloc[i:]]).reset_index(drop=True)
                ### 2. Insert the missed column into new_matrix:
                new_matrix.insert(i, 'new_'+ str(i), 0)
            else:
                continue
        new_matrix.columns = list(new_feature_names[0])                 ## NEW: Add feature names as column names
    else:
        new_matrix.columns = list(new_feature_names[0])                 ## NEW: Add feature names as column names
    return new_matrix, new_feature_names                                ## Returns the parsed down df and word/var df



#####################
 # Call the parse_sparse_matrix function for each variables's spare matrix.
  ## TO DO: 
      # Create a list of the words you want to reduce down to.
      # Have the function save to a new sparse matrix and a new pandas dataframe of the features and the main variable
      # Call the (a) list of words/grams to limit the matrix (b) the name of the vect that was fit (usually 'vect' or 'vect_2', etc), (c) the name of the sparse matrix (usually doc_term_matrix) (d) the name of the variable in quotes ('XX') to identify which word goes with which variable
#####################
####### These were the words with higher feature_importance in the model with ALL words
#### DTITLE
dtitle_list = ['and', 'case', 'case report', 'editor', 'extract']
doc_term_matrix, aa_words = parse_sparse_matrix(dtitle_list, vect, doc_term_matrix, 'DTITLE')
#### SHORT_TITLE
short_title_list = ["and", "in", "of", "exercise", "experiments"]
doc_term_matrix_2, bb_words = parse_sparse_matrix(short_title_list, vect_2, doc_term_matrix_2, 'SHORT_TITLE')
#### ABSTRACT_TEXT
abstract_list = ['abstract', 'across', 'acupuncture', 'addition of', 'administrated', 'adjusted', 'all the', 'an']
doc_term_matrix_3, cc_words = parse_sparse_matrix(abstract_list, vect_3, doc_term_matrix_3, 'ABSTRACT_TEXT')
#### DEPARTMENT
department_list = ["mathematics", "of", "school of", "and"]
doc_term_matrix_4, dd_words = parse_sparse_matrix(department_list, vect_4, doc_term_matrix_4, 'DEPARTMENT')
#### INSTITUTE
institute_list = ["medical", "hospital"]
doc_term_matrix_5, ee_words = parse_sparse_matrix(institute_list, vect_5, doc_term_matrix_5, 'INSTITUTE')
#### PTITLE
ptitle_list = ["dr", "mr", "prof"]
doc_term_matrix_6, ff_words = parse_sparse_matrix(ptitle_list, vect_6, doc_term_matrix_6, 'PTITLE')
#### ALL_AUTHOR_CONTRIBUTOR_ROLES
contributor_roles_list = ["acquisition", "conceptualization"]
doc_term_matrix_7, gg_words = parse_sparse_matrix(contributor_roles_list, vect_7, doc_term_matrix_7, 'ALL_AUTHOR_CONTRIBUTOR_ROLES')
#### FUNDER_NAME
funder_name_list = ["institutes of", "and", "de", "for", "of", "foundation"]
doc_term_matrix_8, hh_words = parse_sparse_matrix(funder_name_list, vect_8, doc_term_matrix_8, 'FUNDER_NAME')



#################################################
  ###### Combine the counts and features dataframes (will combine with dummy variables below too)
#################################################
####### Combine the sparse matracies together as one variable list
X_list = pd.concat([doc_term_matrix,doc_term_matrix_2, doc_term_matrix_3, doc_term_matrix_4,doc_term_matrix_5, doc_term_matrix_6,doc_term_matrix_7, doc_term_matrix_8], axis=1)

####### Combine the feature names AND which variable they come from so you know what is what:
full_features = pd.concat([aa_words, bb_words, cc_words, dd_words, ee_words, ff_words, gg_words, hh_words], ignore_index=True, sort=False)




## Create non-free text variables:

In [ ]:
#################################################
  ###### Create variables: Make into dummy variables
#################################################
############################
  #### Not_filled then drop:
############################
####### POSITION
import numpy as np
data['POSITION_not_filled'] = np.where((data['POSITION'].isnull()) | (data['POSITION'] == " ") | (data['POSITION'] == ""), 1, 0)
data = data.drop('POSITION', axis=1)
####### FUNDER_NAME: Turn into 'anything filled in vs. not'
data['FUNDER_NAME_not_filled'] = np.where((data['FUNDER_NAME'].isnull()) | (data['FUNDER_NAME'] == " ") | (data['FUNDER_NAME'] == ""), 1, 0)

data = data.drop('FUNDER_NAME', axis=1)


############################
  #### CATEGORY: DUE TO LOW NUMBERS ON MANY OF THEM, I ONLY USED:
      # 'Research Article'
      # 'Research Articles'
      # Either 'Research Article' or 'Research Articles'
############################
data['CATEGORY_clinical_trial'] = np.where((data['CATEGORY'] == 'Clinical Trial') | (data['CATEGORY'] == 'Clinical trial'), 1, 0)
data['CATEGORY_Research_Article'] = np.where((data['CATEGORY'] == 'Research Article'), 1, 0)
data['CATEGORY_Research_Articles'] = np.where((data['CATEGORY'] == 'Research Articles'), 1, 0)
#data['CATEGORY_RA_or_RAs'] = np.where((data['CATEGORY'] == 'Research Article') | (data['CATEGORY'] == 'Research Articles'), 1, 0)


############################
  #### SECTIONNAME: Create 20 dummy variables OR create specific ones; Also create a 'not_filled'
############################
data['SECTIONNAME_not_filled'] = np.where((data['SECTIONNAME'].isnull()) | (data['SECTIONNAME'] == " ") | (data['SECTIONNAME'] == ""), 1, 0)
#data = pd.get_dummies(data, columns=['SECTIONNAME'])

####### Only using specific SECTIONNAMES with higher/lower importance:
data['SECTIONNAME_Life & Social Sciences'] = np.where((data['SECTIONNAME'] == 'Life & Social Sciences'), 1, 0)
data['SECTIONNAME_Life Sciences'] = np.where((data['SECTIONNAME'] == 'Life Sciences'), 1, 0)
data['SECTIONNAME_Other'] = np.where((data['SECTIONNAME'] == 'Other'), 1, 0)
data['SECTIONNAME_Medicine and Health Sciences'] = np.where((data['SECTIONNAME'] == 'Medicine and Health Sciences'), 1, 0)
data['SECTIONNAME_SECTIONNAME_Medicine and public health'] = np.where((data['SECTIONNAME'] == 'SECTIONNAME_Medicine and public health'), 1, 0)
data['SECTIONNAME_Environmental Sciences'] = np.where((data['SECTIONNAME'] == 'Environmental Sciences'), 1, 0)
data['SECTIONNAME_Clinical'] = np.where((data['SECTIONNAME'] == 'Clinical'), 1, 0)
data['SECTIONNAME_Physical sciences and engineering'] = np.where((data['SECTIONNAME'] == 'Physical sciences and engineering'), 1, 0)
data['SECTIONNAME_Clinical Sciences'] = np.where((data['SECTIONNAME'] == 'Clinical Sciences'), 1, 0)
data['SECTIONNAME_Applied Mathematics'] = np.where((data['SECTIONNAME'] == 'Applied Mathematics'), 1, 0)
data['SECTIONNAME_Earth Sciences'] = np.where((data['SECTIONNAME'] == 'Earth Sciences'), 1, 0)
data['SECTIONNAME_Social and Behavioral Sciences'] = np.where((data['SECTIONNAME'] == 'Social and Behavioral Sciences'), 1, 0)




############################
  #### COUNTRY: 'anything filled in vs. not' ... Dummy variables for these: PAKISTAN, ETHIOPIA, INDIA, CHINA, UNITED STATES
    # NOTE: You could also try out EVERY country: just use the same code as SECTIONNAME right above
############################
####### Using ALL countries as separate dummy variables:
#data['COUNTRY_not_filled'] = np.where(data[['COUNTRY']].isnull(), 1, 0)
#data = pd.get_dummies(data, columns=['COUNTRY'])

####### Only using specific countries with higher/lower importance::
data['COUNTRY_not_filled'] = np.where(data[['COUNTRY']].isnull(), 1, 0)

data['COUNTRY_AUSTRALIA'] = np.where((data['COUNTRY'] == 'AUSTRALIA'), 1, 0)
data['COUNTRY_BANGLADESH'] = np.where((data['COUNTRY'] == 'BANGLADESH'), 1, 0)
data['COUNTRY_CANADA'] = np.where((data['COUNTRY'] == 'CANADA'), 1, 0)
data['COUNTRY_CHINA'] = np.where((data['COUNTRY'] == 'CHINA'), 1, 0)
data['COUNTRY_EGYPT'] = np.where((data['COUNTRY'] == 'EGYPT'), 1, 0)
data['COUNTRY_ETHIOPIA'] = np.where((data['COUNTRY'] == 'ETHIOPIA'), 1, 0)
data['COUNTRY_GERMANY'] = np.where((data['COUNTRY'] == 'GERMANY'), 1, 0)
data['COUNTRY_INDIA'] = np.where((data['COUNTRY'] == 'INDIA'), 1, 0)
data['COUNTRY_IRAN, ISLAMIC REPUBLIC OF'] = np.where((data['COUNTRY'] == 'IRAN, ISLAMIC REPUBLIC OF'), 1, 0)
data['COUNTRY_MALAYSIA'] = np.where((data['COUNTRY'] == 'MALAYSIA'), 1, 0)
data['COUNTRY_NETHERLANDS'] = np.where((data['COUNTRY'] == 'NETHERLANDS'), 1, 0)
data['COUNTRY_NIGERIA'] = np.where((data['COUNTRY'] == 'NIGERIA'), 1, 0)
data['COUNTRY_PAKISTAN'] = np.where((data['COUNTRY'] == 'PAKISTAN'), 1, 0)
data['COUNTRY_SAUDI ARABIA'] = np.where((data['COUNTRY'] == 'SAUDI ARABIA'), 1, 0)
data['COUNTRY_UNITED KINGDOM'] = np.where((data['COUNTRY'] == 'UNITED KINGDOM'), 1, 0)
data['COUNTRY_UNITED STATES'] = np.where((data['COUNTRY'] == 'UNITED STATES'), 1, 0)




#################################################
  ###### Create variables: Varied:
#################################################
############################
  #### DEPARTMENT: (a) Has only one word (b) not filled in
############################
### One word:
f = data[data['DEPARTMENT'].astype(str).str.contains(" ")]
data['DEPARTMENT_over_one_word'] = np.where(data.index.isin(f.index), 1, 0)
### Not filled in:
data['DEPARTMENT_not_filled'] = np.where((data['DEPARTMENT'].isnull()) | (data['DEPARTMENT'] == ""), 1, 0)

####### Create a ratio variable of the number of Characters (ratio):
temp = []
for i in range(len(data)):
    a = len(str(data['DEPARTMENT'][i]))
    temp.append(a)
data['DEPARTMENT_num_characters'] = temp



############################
  #### ALLAUTHORS: Count the number of semicolons + 1. Use as ratio but also turn into a binned (dummy) variable
############################
####### Create a ratio variable of the number of authors (minus 1):
temp = []
for i in range(len(data)):
    a = str(data['ALLAUTHORS'][i]).count(';')
    temp.append(a)
data['ALLAUTHORS_ratio_count'] = temp

####### Some have a ton of authors so bin instead: 1; 2-3; 4-5; 6-7; 8-10; 11-14; 15-20; 21-35; 35+
############# ONLY USE 2-3 AND RATIO
data['ALLAUTHORS_1'] = np.where((data['ALLAUTHORS_ratio_count'] == 1), 1, 0)
data['ALLAUTHORS_2_3'] = np.where((data['ALLAUTHORS_ratio_count'] == 2) | (data['ALLAUTHORS_ratio_count'] == 3), 1, 0)


############################
  #### INSTITUTE: Anything filled in vs. not' THEN number of characters (ratio) THEN number of words (ratio) THEN Words over 4 THEN words over 6
############################
####### Create variable: INSTITUTE not filled in:
data['INSTITUTE_not_filled'] = np.where((data['INSTITUTE'].isnull()) | (data['INSTITUTE'] == " ") | (data['INSTITUTE'] == ""), 1, 0)

####### Create a ratio variable of the number of Characters (ratio):
temp = []
for i in range(len(data)):
    a = len(str(data['INSTITUTE'][i]))
    temp.append(a)
data['INSTITUTE_num_characters'] = temp


############################
  #### ORIGINAL_SUBMISSION_START_DATE: Dummy variables for each day of the week; Dummy variable for each month:
############################
####### Create date only variable from ORIGINAL_SUBMISSION_START_DATE:
from datetime import datetime
temp = []
for i in range(len(data)):
    a = datetime.strptime(str(data['ORIGINAL_SUBMISSION_START_DATE'][i]).split(' ')[0], '%Y-%m-%d')
    temp.append(a)
data['date'] = temp
data.tail()

####### Create variables for each day of the week:
  # NOTE: Monday is 0 and Sunday is 6.
temp = []
for i in range(len(data)):
    a = data['date'][i].weekday()
    temp.append(a)
data['dayofweek'] = temp
#### Make dummy variables:
data['dayofweek_mon'] = np.where((data['dayofweek'] == 0), 1, 0)
data['dayofweek_tue'] = np.where((data['dayofweek'] == 1), 1, 0)
data['dayofweek_wed'] = np.where((data['dayofweek'] == 2), 1, 0)
data['dayofweek_thur'] = np.where((data['dayofweek'] == 3), 1, 0)
data['dayofweek_fri'] = np.where((data['dayofweek'] == 4), 1, 0)
data['dayofweek_sat'] = np.where((data['dayofweek'] == 5), 1, 0)
data['dayofweek_sun'] = np.where((data['dayofweek'] == 6), 1, 0)

data = data.drop('dayofweek', axis=1)


####### Create dummy variables for each month:
temp = []
for i in range(len(data)):
    a = data['date'][i].month
    temp.append(a)
data['month'] = temp

data['month_1'] = np.where((data['month'] == 1), 1, 0)
data['month_2'] = np.where((data['month'] == 2), 1, 0)
data['month_3'] = np.where((data['month'] == 3), 1, 0)
data['month_4'] = np.where((data['month'] == 4), 1, 0)
data['month_5'] = np.where((data['month'] == 5), 1, 0)
data['month_6'] = np.where((data['month'] == 6), 1, 0)
data['month_7'] = np.where((data['month'] == 7), 1, 0)
data['month_8'] = np.where((data['month'] == 8), 1, 0)
data['month_9'] = np.where((data['month'] == 9), 1, 0)
data['month_10'] = np.where((data['month'] == 10), 1, 0)
data['month_11'] = np.where((data['month'] == 11), 1, 0)
data['month_12'] = np.where((data['month'] == 12), 1, 0)

data = data.drop('month', axis=1)





#################################################
  ###### Combine pandas data of non-free text into the main I.V. list: the 'X_list' sparse matrix)
#################################################
############################
  #### Accumulate all pandas columns to include in analysis:
############################
####### Make sure to keep these 2 pre-created variables:
data['REQUEST_EDITOR_2'] = np.where((data['REQUEST_EDITOR'] == 1), 1, 0)
data['OPPOSE_REVIEWERS_2'] = np.where((data['OPPOSE_REVIEWERS'] == 1), 1, 0)

##### The 1st variable to include is "POSITION_not_filled", get that position until the end
   # Remove 'date' column too
data = data.drop(['date'], axis=1)
start_col = data.columns.get_loc('POSITION_not_filled')
for_x_list = data.iloc[:, start_col : len(data.columns)]





In [ ]:
#################################################
  ###### Combing ALL free text words and ngrams with the non-free text variables
#################################################
############################
  #### Add the non-free and free text columns:
############################
X_list = pd.concat([X_list, for_x_list], axis=1)

############################
  #### Add on the pandas column names to the 'full_features' list of feature names
############################
for_x_list = pd.DataFrame(list(for_x_list.columns))
for_x_list['main_var'] = 'non_text'
full_features = pd.concat([pd.DataFrame(full_features), for_x_list], ignore_index=True, sort=False)


############################
  #### Turn variables into the appropriate format (esp the free text based vars)
############################
for i in range(len(X_list.columns)):
    if str(X_list.iloc[:,i].dtypes) == 'Sparse[int64, 0]':
        X_list.iloc[:,i] = X_list.iloc[:,i].sparse.to_dense().astype('int64')
    else:
        X_list.iloc[:,i] = X_list.iloc[:,i].astype('int64')


############################
  #### Add on the pandas column names to the 'full_features' list of feature names
############################
for_x_list = pd.DataFrame(list(for_x_list.columns))
for_x_list['main_var'] = 'non_text'
full_features = pd.concat([pd.DataFrame(full_features), for_x_list], ignore_index=True, sort=False)



############################
  #### bigrams are seen as Spare dtypes, turn them into int64 only.
############################
test = str(X_list[X_list.columns[2]].dtypes)
for i in X_list.columns:
    if str(X_list[i].dtypes) == test:
        X_list[i] = X_list[i].sparse.to_dense()
    else:
        X_list[i] = X_list[i].astype('int64')


## Load in the model and make predictions

In [ ]:
#################################################
  ###### Create and save the XGBClassifier tree iterations:
#################################################
############################
  #### load the 2012 and 2017 model from disk
############################
import pickle
filename = '6_8_desk_reject_model/2012_6_8_model.sav'
model_2012 = pickle.load(open(filename, 'rb'))

filename = '6_8_desk_reject_model/2017_6_8_model.sav'
model_2017 = pickle.load(open(filename, 'rb'))


############################
  #### Give 'X_list' dummy column names to match model:
############################
feature_list = []
for i in range(328):
    feature_list.append('f'+ str(i))
aa = X_list.copy()
aa.columns = feature_list



############################
  #### Connect predictions to the PUBDNUMBER (AKA the ManuscriptNumber) and DOCUMENTID
############################
prediction_data = data[['PUBDNUMBER', 'DOCUMENTID']]


############################
  #### Make predictions per model per each threshold:
    # NOTE: See supplemental for methods on trying to figure out threshold groupings to make likelihood score
############################

####### Create predictions for the 2012 model with the relevant thresholds
threshold_2012 = [.04, .11, .25, .53, .70]
count = 0
for i in threshold_2012:
    prediction_data['p_2012_' + str(count)] = (model_2012.predict_proba(aa)[:,1] >= i).astype('int')
    count += 1

####### Create predictions for the 2017 model with the relevant thresholds
threshold_2017 = [.03, .11, .21, .46, .60]
count = 0
for i in threshold_2017:
    prediction_data['p_2017_' + str(count)] = (model_2017.predict_proba(aa)[:,1] >= i).astype('int')
    count += 1
    



############################
  #### Create the desk reject liklihood score based on above model predictions:
############################
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_0'] == 1) | (prediction_data['p_2017_0'] == 1), 1, 0)
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_0'] == 1) & (prediction_data['p_2017_0'] == 1), 2, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_1'] == 1) | (prediction_data['p_2017_1'] == 1), 3, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_1'] == 1) & (prediction_data['p_2017_1'] == 1), 4, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_2'] == 1) | (prediction_data['p_2017_2'] == 1), 5, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_2'] == 1) & (prediction_data['p_2017_2'] == 1), 6, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_3'] == 1) | (prediction_data['p_2017_3'] == 1), 7, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_3'] == 1) & (prediction_data['p_2017_3'] == 1), 8, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_4'] == 1) | (prediction_data['p_2017_4'] == 1), 9, prediction_data['desk_reject_liklihood_score'])
prediction_data['desk_reject_liklihood_score'] = np.where((prediction_data['p_2012_4'] == 1) & (prediction_data['p_2017_4'] == 1), 10, prediction_data['desk_reject_liklihood_score'])


############################
  #### Remove unneeded columns:
############################
prediction_data = prediction_data[['PUBDNUMBER', 'DOCUMENTID', 'desk_reject_liklihood_score']]


############################
  #### Save prediction table to file:
############################
prediction_data.to_csv('NEW_desk_reject_predictions.csv', index = False)


In [ ]:
#################################################
  ###### SUPPLMENTAL: Comparing scores of both models at different thresholds to figure out liklihood scoring system
#################################################
    
############################
  #### Make predictions per model per a given threshold: This example is for threshold = 30:
############################
#p_2012_30 = (model_2012.predict_proba(aa)[:,1] >= 0.03).astype('int')
#p_2017_30 = (model_2017.predict_proba(aa)[:,1] >= 0.03).astype('int')
#print("percent tagged as desk reject, 30 threshold, 2012 model", sum(p_2012_30) / len(p_2012_30))
#print("percent tagged as desk reject, 30 threshold, 2017 model: ", sum(p_2017_30) / len(p_2017_30))



############################
  #### Connect predictions for both models to the MS number
############################
#prediction_data = prediction_data[['PUBDNUMBER', 'DOCUMENTID']]
#prediction_data['predict_30_2012'] = p_2012_30
#prediction_data['predict_30_2017'] = p_2017_30
#prediction_data

#################################################
  #### Bring in Desk reject data and compare actual to predicted with threshold = 30
#################################################
#import numpy as np
#dv = pd.read_csv("April_to_May_test/april_may_desk_rejects.csv")
#prediction_data = pd.merge(prediction_data, dv[['ManuscriptNumber', 'Date_Submit']], how='left', left_on = ['PUBDNUMBER'], right_on = ['ManuscriptNumber'])
#prediction_data['desk_reject'] = np.where(prediction_data[['Date_Submit']].isnull(), 0, 1)
#prediction_data.drop(['ManuscriptNumber', 'Date_Submit'], axis = 1)


#################################################
  ###### Do several types of accuracy checks on actual vs predicted with threshold = 30
#################################################
 # precision: lower means it's more likely to get guess something is 0 and then that guess will be wrong.
 # recall: lower means it's more likely to NOT guess something is 0 when it actually is 0.
    # So the recall the desk rejects is quite low.
 # f-1 score is an overall score of precision and recall.
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#print("*************** 2012 Model:")
#print(confusion_matrix(prediction_data['desk_reject'], prediction_data['predict_30_2012']), "\n")
#print(classification_report(prediction_data['desk_reject'], prediction_data['predict_30_2012']), "\n")
#print(accuracy_score(prediction_data['desk_reject'], prediction_data['predict_30_2012']))

#print("\n*************** 2017 Model:")
#print(confusion_matrix(prediction_data['desk_reject'], prediction_data['predict_30_2017']), "\n")
#print(classification_report(prediction_data['desk_reject'], prediction_data['predict_30_2017']), "\n")
#print(accuracy_score(prediction_data['desk_reject'], prediction_data['predict_30_2017']))




In [ ]:
#################################################
  ###### SUPPLEMENTAL: Use the below to figure out the proper thresholds to use and when for likelihood scores
#################################################

####### Define the model as either model_2012 or model_2017
#model_to_use = model_2012



############################
  #### Change around the threshold and getting performance metrics, sorted by f1-score:
############################
#from sklearn.metrics import f1_score, recall_score, precision_score
#fone = {}
#for i in range(3, 75):
#    threshold = i / 100  # use this to optimize
#    predicted_proba = model_to_use.predict_proba(aa)
#    y_pred = (predicted_proba [:,1] >= threshold).astype('int')
#    fone[i] = round(f1_score(prediction_data['desk_reject'], y_pred),3), round(recall_score(prediction_data['desk_reject'], y_pred),3), round(precision_score(prediction_data['desk_reject'], y_pred),3), round(accuracy_score(prediction_data['desk_reject'], y_pred),3), round(sum(y_pred)/len(y_pred),3), round(sum(prediction_data['desk_reject']) / len(prediction_data['desk_reject']),3)

####### Print out scores of thresholds:
#tt = sorted(fone.items(), key=lambda x: x[1], reverse = True)
#print("Threshold,  f1-Score,  Recall,  Precision score, Model Accuracy, Percent Tagged as desk reject, Percent actual reject      ...  Below are sorted by f1 score")
#tt




####################################################
#################################################### 
  ### SUPPLEMENTAL NOTES: From the stats above I created the following break downs:
####################################################
####################################################

############################
  #### Notes on the models at different thresholds:
############################
    
####### 2012 Model: Thresholds and what they mean
  # 33 is the correct percentage of tagged desk rejects.
  # 71 is when percentage of tagged desk rejects first gets under 1%
  # 9 is when percentage of tagged desk rejects first gets under 50%
  # 22 gets a recall over 50%
  # 12 or 13 is best f1 score
  # 53 or 54 is when the increase in overal model fit becomes negligible.
  # 3 is a recal of 95%, 4 is 92.1%, 5 is 88.8, 6 is 87.3, 7 is 84.4
  # The precision is never great but it's best at 66

####### 2017 Model: Thresholds and what they mean
  # 33 is the correct percentage of tagged desk rejects.
  # 73 is when percentage of tagged desk rejects first gets under 1%
  # 9 is when percentage of tagged desk rejects first gets under 50%
  # 23 gets a recall over 50%
  # 16 or 18 is best f1 score
  # 49 is when the increase in overal model fit becomes negligible.
  # 3 is a recal of 93.4%, 4 is 91.9%, 5 is 89.9, 6 is 89, 7 is 86
  # The precision is never great but it's best at 55, generally between 49 and 58



############################
  #### Thresholds I will use:
############################
############# 2012 model threshold categories:
  # 0 = Not tagged with a 4 threshold
  # 1 = Tagged with a 4 threshold but not tagged with a 11 threshold
  # 2 = Tagged with a 11 threshold but not tagged with a 25 threshold
  # 3 = Tagged with a 25 threshold but not tagged with a 53 threshold
  # 4 = Tagged with a 53 threshold but not tagged with a 70 threshold
  # 5 = Tagged with a 70 threshold
############# 2017 model threshold categories:
  # 0 = Not tagged with a 3 threshold
  # 1 = Tagged with a 3 threshold but not tagged with a 11 threshold
  # 2 = Tagged with a 11 threshold but not tagged with a 21 threshold
  # 3 = Tagged with a 21 threshold but not tagged with a 46 threshold
  # 4 = Tagged with a 46 threshold but not tagged with a 60 threshold
  # 5 = Tagged with a 60 threshold

#threshold_2012 = [4, 11, 25, 53, 70]
#threshold_2017 = [3, 11, 21, 46, 60]

